# Connect + Refresh Drop Creds

In [1]:
import os
# IMPORTANT: set this BEFORE importing app
os.environ["DATABASE_URL"] = "postgresql://localhost/vibedrop_dev"

from app import app
from models import db, User
from services.scoring import snapshot_user_all_versions
from sqlalchemy import text
import pandas as pd

def sql_df(q: str, **params) -> pd.DataFrame:
    with app.app_context():
        with db.engine.connect() as conn:
            return pd.read_sql(text(q), conn, params=params)

# sanity
with app.app_context():
    print("Flask-SQLAlchemy DB URL:", str(db.engine.url))
    print(db.session.execute(text("select current_database() as db, current_schema() as schema;")).mappings().one())

Flask-SQLAlchemy DB URL: postgresql://localhost/vibedrop_dev
{'db': 'vibedrop_dev', 'schema': 'public'}


In [2]:
# Pick your versions (ensure they exist in services.scoring._apply_formula)
VERSIONS = (1, 2, 3, 4)  # change to (1,2,3) if v4 isn't present locally

with app.app_context():
    user_ids = [uid for (uid,) in db.session.query(User.id).all()]
    replaced = 0
    for uid in user_ids:
        # replace=True ensures exactly one latest row per (user,version)
        rows = snapshot_user_all_versions(uid, versions=VERSIONS, replace=True, commit=False)
        replaced += len(rows)
    db.session.commit()
print(f"Refreshed {len(user_ids)} users × {len(VERSIONS)} versions = {replaced} rows")

Refreshed 21 users × 4 versions = 84 rows


In [3]:
display(sql_df("""
WITH latest_per_version AS (
  SELECT DISTINCT ON (user_id, score_version)
         user_id, score_version, drop_cred_score,
         total_likes, total_dislikes, total_possible, computed_at
  FROM drop_creds
  ORDER BY user_id, score_version, computed_at DESC
),
latest_totals AS (
  SELECT DISTINCT ON (user_id)
         user_id, total_likes, total_dislikes, total_possible, computed_at
  FROM latest_per_version
  ORDER BY user_id, computed_at DESC
)
SELECT
  t.user_id,
  t.total_likes,
  t.total_dislikes,
  t.total_possible,
  MAX(CASE WHEN v.score_version = 1 THEN v.drop_cred_score END) AS score_v1,
  MAX(CASE WHEN v.score_version = 2 THEN v.drop_cred_score END) AS score_v2,
  MAX(CASE WHEN v.score_version = 3 THEN v.drop_cred_score END) AS score_v3,
  MAX(CASE WHEN v.score_version = 4 THEN v.drop_cred_score END) AS score_v4
FROM latest_totals t
JOIN latest_per_version v USING (user_id)
GROUP BY t.user_id, t.total_likes, t.total_dislikes, t.total_possible
ORDER BY score_v4;
"""))

,user_id,total_likes,total_dislikes,total_possible,score_v1,score_v2,score_v3,score_v4
0,22,0,0,0,3.8,4.3,2.8,2.5
1,11,0,0,0,3.8,4.3,2.8,2.5
2,20,0,0,0,3.8,4.3,2.8,2.5
3,21,0,0,0,3.8,4.3,2.8,2.5
4,8,0,1,45,3.8,4.1,3.3,3.4
5,7,0,0,30,3.8,4.3,3.3,3.4
6,19,0,0,7,3.8,4.3,4.2,3.8
7,9,0,2,15,3.8,2.6,3.7,3.8
8,12,0,2,7,3.8,0.3,4.2,3.8
9,3,1,0,45,4.1,4.6,4.2,4.3


### v3: Bayesian smoothing (Beta-Binomial posterior mean)

We assume likes $L$ out of $N$ possible ratings. With prior $\text{Beta}(\alpha, \mu \alpha)$, the posterior mean score is

$$
\text{Score}_{v3} = \frac{L + \mu \alpha}{N + \alpha}.
$$

### v4a: Bayesian smoothing + participation bonus

We add a bonus term proportional to the number of submissions $S$, capped at $S_{\max}$:

$$
\text{Score}_{v4a} = \frac{L + \mu \alpha}{N + \alpha} \;+\; \beta \cdot \min\!\left(\frac{S}{S_{\max}}, 1\right).
$$

### v4b: Convex combination of accuracy and participation

Here we interpolate between the Bayesian score (accuracy) and raw participation:

$$
\text{Score}_{v4b} = (1 - \lambda)\,\frac{L + \mu \alpha}{N + \alpha} \;+\; \lambda \cdot S,
$$

where $\lambda \in [0,1]$ controls the tradeoff.

# Ad-hoc cells (run per session)
### The above cells should always remain unchanged and should be ran every time this notebook is started

In [4]:
# load latest stats per user (lowercase aliases)
latest = sql_df("""
WITH latest_per_version AS (
  SELECT DISTINCT ON (user_id, score_version)
         user_id, score_version, drop_cred_score,
         total_likes, total_dislikes, total_possible, computed_at
  FROM drop_creds
  ORDER BY user_id, score_version, computed_at DESC
),
latest_totals AS (
  SELECT DISTINCT ON (user_id)
         user_id, total_likes, total_dislikes, total_possible, computed_at
  FROM latest_per_version
  ORDER BY user_id, computed_at DESC
)
SELECT 
  user_id, 
  total_likes   AS l,
  total_dislikes AS d,
  total_possible AS p
FROM latest_totals
ORDER BY user_id;
""")
subs = sql_df("""
  SELECT user_id, COUNT(*) AS s
  FROM submissions
  GROUP BY user_id
""")

latest = latest.merge(subs, on="user_id", how="left").fillna({"s": 0})
latest[["user_id","l","d","p","s"]].head()

,user_id,l,d,p,s
0,2,1,1,43,7.0
1,3,1,0,45,3.0
2,4,1,0,30,2.0
3,5,4,1,30,2.0
4,6,2,0,45,3.0


In [5]:
def score_v3(L, P, alpha=10.0, mu=0.1):
    P = max(P, 0)
    return 10.0 * (L + alpha*mu) / (P + alpha) if (P + alpha) > 0 else 10.0 * mu

def score_v4a(L, P, S, alpha=10.0, mu=0.1, beta=0.05, Smax=10):
    base = score_v3(L, P, alpha=alpha, mu=mu)
    boost = beta * min(S, Smax)
    return base + boost

def score_v4b(L, P, S, lam=0.7):
    term1 = (L / P) if P > 0 else 0.0
    term2 = (L / S) if S > 0 else 0.0
    return 10.0 * (lam*term1 + (1-lam)*term2)

In [6]:
latest = latest.copy()
# ensure s exists (if you skipped the submissions join, you can approximate s=p, but better to join)
if "s" not in latest.columns:
    latest["s"] = latest["p"]  # fallback; preferably use the submissions join above

latest["v3_test"]  = latest.apply(lambda r: score_v3(r.l, r.p, alpha=10, mu=0.1), axis=1)
latest["v4a_test"] = latest.apply(lambda r: score_v4a(r.l, r.p, r.s, alpha=10, mu=0.1, beta=0.05, Smax=10), axis=1)
latest["v4b_test"] = latest.apply(lambda r: score_v4b(r.l, r.p, r.s, lam=0.7), axis=1)

latest.head()

,user_id,l,d,p,s,v3_test,v4a_test,v4b_test
0,2,1,1,43,7.0,0.377358,0.727358,0.591362
1,3,1,0,45,3.0,0.363636,0.513636,1.155556
2,4,1,0,30,2.0,0.500000,0.600000,1.733333
3,5,4,1,30,2.0,1.250000,1.350000,6.933333
4,6,2,0,45,3.0,0.545455,0.695455,2.311111


In [7]:
# Learn global priors using the "latest" df you already built with cols: user_id, l, d, p, s
L_total = latest["l"].sum()
P_total = latest["p"].sum()
mu_global = (L_total / P_total) if P_total > 0 else 0.1

S_median = float(latest["s"].median()) if len(latest) else 0
S_p90    = float(latest["s"].quantile(0.90)) if len(latest) else 0

print(f"μ_global ≈ {mu_global:.3f}   (overall like rate)")
print(f"Submissions: median={S_median:.0f}  p90={S_p90:.0f}")

μ_global ≈ 0.051   (overall like rate)
Submissions: median=1  p90=3


In [8]:
# Sweep params for v3 (Bayesian) and v4 (participation-friendly)
from itertools import product

def sweep_v3(df, alphas=(5,10,20,40), mus=None):
    if mus is None:
        mus = (max(0.01, mu_global*0.5), mu_global, min(0.5, mu_global*1.5))
    rows = []
    for a, m in product(alphas, mus):
        sc = df.apply(lambda r: score_v3(r.l, r.p, alpha=a, mu=m), axis=1)
        rows.append({
            "alpha": a, "mu": round(m,3),
            "mean": sc.mean(), "std": sc.std(ddof=0),
            "min": sc.min(), "max": sc.max()
        })
    return pd.DataFrame(rows).sort_values(["alpha","mu"])

def sweep_v4a(df, alphas=(10,), mus=None, betas=(0.03,0.05,0.07), smaxs=None):
    if mus is None:
        mus = (mu_global,)
    if smaxs is None:
        Smax_guess = int(max(5, min(10, S_p90 or 10)))
        smaxs = (Smax_guess, Smax_guess+5)
    rows = []
    for a,m,b,sm in product(alphas, mus, betas, smaxs):
        sc = df.apply(lambda r: score_v4a(r.l, r.p, r.s, alpha=a, mu=m, beta=b, Smax=sm), axis=1)
        rows.append({
            "alpha": a, "mu": round(m,3), "beta": b, "Smax": sm,
            "mean": sc.mean(), "std": sc.std(ddof=0),
            "min": sc.min(), "max": sc.max()
        })
    return pd.DataFrame(rows).sort_values(["beta","Smax"])

def sweep_v4b(df, lams=(0.3,0.5,0.7,0.9)):
    rows = []
    for lam in lams:
        sc = df.apply(lambda r: score_v4b(r.l, r.p, r.s, lam=lam), axis=1)
        rows.append({
            "lam": lam,
            "mean": sc.mean(), "std": sc.std(ddof=0),
            "min": sc.min(), "max": sc.max()
        })
    return pd.DataFrame(rows).sort_values("lam")

g3  = sweep_v3(latest)
g4a = sweep_v4a(latest)
g4b = sweep_v4b(latest)

display(g3)
display(g4a)
display(g4b)

,alpha,mu,mean,std,min,max
0,5,0.025,0.518158,0.495858,0.025316,1.772152
1,5,0.051,0.616620,0.500965,0.050633,1.877637
2,5,0.076,0.715081,0.519563,0.075949,1.983122
3,10,0.025,0.450159,0.359866,0.046030,1.325391
4,10,0.051,0.574417,0.366526,0.092060,1.474311
5,10,0.076,0.698675,0.387757,0.138090,1.623232
6,20,0.025,0.388277,0.247517,0.077897,0.928270
7,20,0.051,0.542707,0.254837,0.155794,1.115799
8,20,0.076,0.697136,0.276497,0.233690,1.303329
9,40,0.025,0.338118,0.159460,0.119136,0.716094


,alpha,mu,beta,Smax,mean,std,min,max
0,10,0.051,0.03,5,0.620131,0.359048,0.182060,1.504311
1,10,0.051,0.03,10,0.622989,0.357793,0.182060,1.504311
2,10,0.051,0.05,5,0.650608,0.356070,0.226582,1.524311
3,10,0.051,0.05,10,0.655369,0.355149,0.226582,1.524311
4,10,0.051,0.07,5,0.681084,0.354748,0.266582,1.544311
5,10,0.051,0.07,10,0.687750,0.355120,0.266582,1.544311


,lam,mean,std,min,max
0,0.3,4.427255,5.110502,0.0,14.857143
1,0.5,3.365154,3.882836,0.0,11.428571
2,0.7,2.303052,2.686117,0.0,8.000000
3,0.9,1.240950,1.591745,0.0,5.500000


In [9]:
# Compare candidate rankings with what you have now
# pull current v1–v4 from DB
current = sql_df("""
WITH latest_per_version AS (
  SELECT DISTINCT ON (user_id, score_version)
         user_id, score_version, drop_cred_score
  FROM drop_creds
  ORDER BY user_id, score_version, computed_at DESC
)
SELECT user_id,
  MAX(CASE WHEN score_version=1 THEN drop_cred_score END) AS v1,
  MAX(CASE WHEN score_version=2 THEN drop_cred_score END) AS v2,
  MAX(CASE WHEN score_version=3 THEN drop_cred_score END) AS v3,
  MAX(CASE WHEN score_version=4 THEN drop_cred_score END) AS v4
FROM latest_per_version
GROUP BY user_id
ORDER BY user_id;
""")

df = latest.merge(current, on="user_id", how="left")

# choose a v3 candidate
cand_v3 = df.apply(lambda r: score_v3(r.l, r.p, alpha=10, mu=mu_global), axis=1)
# choose a v4a candidate
cand_v4a = df.apply(lambda r: score_v4a(r.l, r.p, r.s, alpha=10, mu=mu_global, beta=0.05, Smax=int(max(5, min(10, S_p90 or 10)))) , axis=1)

eval_df = df.assign(cand_v3=cand_v3, cand_v4a=cand_v4a)
corrs = eval_df[["v1","v2","v3","v4","cand_v3","cand_v4a"]].rank().corr(method="spearman")
display(corrs)

,v1,v2,v3,v4,cand_v3,cand_v4a
v1,1.000000,0.980038,0.949762,0.959440,0.790233,0.891562
v2,0.980038,1.000000,0.937150,0.925233,0.814943,0.862596
v3,0.949762,0.937150,1.000000,0.989472,0.689228,0.777928
v4,0.959440,0.925233,0.989472,1.000000,0.672352,0.796328
cand_v3,0.790233,0.814943,0.689228,0.672352,1.000000,0.937132
cand_v4a,0.891562,0.862596,0.777928,0.796328,0.937132,1.000000
